<a href="https://colab.research.google.com/github/Man-snow/llm2025compet_Man-snow/blob/main/Math%20Evol-Instruct/BigMath%20with%20DeepSeek-R1-0528/generate_problems_Qwen2_5_1_5B_Instruct_AWQ.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install vllm==0.5.1 datasets pandas "transformers>=4.42.0"

In [14]:
# !!修正!!: login()ヘルパーを使わず、トークンを直接変数に格納する方式に変更
import os
import time
import pandas as pd
from datasets import load_dataset, Dataset
from vllm import LLM, SamplingParams
import re
from transformers import AutoTokenizer
from IPython.display import display
import getpass

# このセルを実行すると表示される入力ボックスに、「Write」権限を持つトークンを貼り付けてください。
# 入力した文字は表示されませんが、正常に入力されています。
hf_write_token = getpass.getpass("Hugging Face Write Token:")
print("Token received.")

Hugging Face Write Token:··········
Token received.


In [12]:
# --- 定数設定 ---
MODEL_ID = "Qwen/Qwen2.5-1.5B-Instruct-AWQ"
SOURCE_DATASET_ID = "SynthLabsAI/Big-Math-RL-Verified"
# !!修正!!: Hugging Face Hubのアップロード先リポジトリIDを指定
OUTPUT_DATASET_ID = "Man-snow/test_evolved-math-problems"

# 問題を上方修正するためのプロンプト
UPWARD_EVOLUTION_PROMPT_TEMPLATE = """
You are an expert in creating complex mathematical problems. Your task is to rewrite the given instruction to make it more challenging.

#Instruction#
{problem}

Follow these steps precisely.
Step 1: Understand the core concept and structure of the "#Instruction#". Identify the key elements such as variables, conditions, participants, actions, or processes that can be manipulated to increase complexity. Also, recognize the theme of the instruction and ensure it remains consistent throughout the evolution.
Step 2: Formulate a comprehensive plan to increment the complexity of the "#Instruction#" based on the identified elements in Step 1. The plan should involve modifying or expanding at least three components from the list. It is crucial to ensure that all components in the instruction are logically interconnected and that the complexity increase is coherent and justified. The plan should avoid introducing variables or conditions without clear criteria for determining their values or without contributing to the overall complexity. In this step, consider adding more real-world constraints and dependencies between variables to make the problem more challenging. And you can also add more constraints, concretizing, increasing reasoning.
Step 3: Implement the plan step by step to create the "#Rewritten Instruction#". Ensure the rewritten instruction maintains a logical sequence and avoids ambiguity or confusion. If additional variables or conditions are introduced, provide clear and unambiguous methods or criteria for determining their values. The "#Rewritten Instruction#" should not exceed the original "#Instruction#" by more than 30 words to ensure readability and comprehension.
Step 4: Review the "#Rewritten Instruction#" thoroughly to identify any unreasonable elements or inconsistencies. Make sure the "#Rewritten Instruction#" is a more complex version of the "#Instruction#". and that it accurately reflects the intended increase in complexity. Adjust any part of the instruction that may lead to misunderstanding or ambiguity, and provide the "#Finally Rewritten Instruction#" without any supplementary explanation.
Please reply strictly in the following format:
Step 1
#Elements Identified#:
...
Step 2
#Plan#:
...
Step 3
#Rewritten Instruction#:
...
Step 4
#Finally Rewritten Instruction#:
...
"""

# --- ヘルパー関数 ---
def parse_final_instruction(text: str) -> str | None:
    """
    モデルの出力から"#Finally Rewritten Instruction#"の部分を抽出する。
    見つからない場合はNoneを返す。
    """
    match = re.search(r"#Finally Rewritten Instruction#:\s*(.*)", text, re.DOTALL)
    if match:
        return match.group(1).strip()
    return None

print("定数とヘルパー関数の定義が完了しました。")

定数とヘルパー関数の定義が完了しました。


In [4]:
print("--- ステップ1: データセットの準備 ---")
try:
    dataset = load_dataset(SOURCE_DATASET_ID, split="train", trust_remote_code=True)
    df = dataset.to_pandas()
    sorted_df = df.sort_values(by=["llama8b_solve_rate", "problem"], ascending=[True, True])
    problems_to_process = sorted_df.head(100)
    print(f"データセットの準備が完了しました。処理対象: {len(problems_to_process)}問")
except Exception as e:
    print(f"データセットの読み込みに失敗しました: {e}")

--- ステップ1: データセットの準備 ---


README.md:   0%|          | 0.00/6.31k [00:00<?, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/32.2M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/251122 [00:00<?, ? examples/s]

データセットの準備が完了しました。処理対象: 100問


In [5]:
print("--- ステップ2: vLLMモデルの初期化 ---")
try:
    llm = LLM(
        model=MODEL_ID,
        quantization="awq",
        trust_remote_code=True,
        gpu_memory_utilization=0.9
    )
    sampling_params = SamplingParams(temperature=0.7, top_p=0.95, max_tokens=1024)
    print("モデルの初期化が完了しました。")
except Exception as e:
    print(f"モデルの初期化に失敗しました: {e}")

--- ステップ2: vLLMモデルの初期化 ---


config.json:   0%|          | 0.00/838 [00:00<?, ?B/s]

WARNING 07-30 01:14:00 config.py:244] awq quantization is not fully optimized yet. The speed can be slower than non-quantized models.
INFO 07-30 01:14:00 llm_engine.py:169] Initializing an LLM engine (v0.5.1) with config: model='Qwen/Qwen2.5-1.5B-Instruct-AWQ', speculative_config=None, tokenizer='Qwen/Qwen2.5-1.5B-Instruct-AWQ', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, rope_scaling=None, rope_theta=None, tokenizer_revision=None, trust_remote_code=True, dtype=torch.float16, max_seq_len=32768, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=awq, enforce_eager=False, kv_cache_dtype=auto, quantization_param_path=None, device_config=cuda, decoding_config=DecodingConfig(guided_decoding_backend='outlines'), observability_config=ObservabilityConfig(otlp_traces_endpoint=None), seed=0, served_model_name=Qwen/Qwen2.5-1.5B-Instruct-AWQ, use_v2_block_manager=False, enable_prefix_cac

tokenizer_config.json: 0.00B [00:00, ?B/s]

vocab.json: 0.00B [00:00, ?B/s]

...

tokenizer.json: 0.00B [00:00, ?B/s]

generation_config.json:   0%|          | 0.00/242 [00:00<?, ?B/s]

INFO 07-30 01:14:05 selector.py:153] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 07-30 01:14:05 selector.py:53] Using XFormers backend.
INFO 07-30 01:14:08 selector.py:153] Cannot use FlashAttention-2 backend for Volta and Turing GPUs.
INFO 07-30 01:14:08 selector.py:53] Using XFormers backend.
INFO 07-30 01:14:09 weight_utils.py:218] Using model weights format ['*.safetensors']


model.safetensors:   0%|          | 0.00/1.61G [00:00<?, ?B/s]

INFO 07-30 01:14:55 weight_utils.py:261] No model.safetensors.index.json found in remote.
INFO 07-30 01:14:56 model_runner.py:255] Loading model weights took 1.1008 GB
INFO 07-30 01:15:01 gpu_executor.py:84] # GPU blocks: 20090, # CPU blocks: 9362
INFO 07-30 01:15:06 model_runner.py:924] Capturing the model for CUDA graphs. This may lead to unexpected consequences if the model is not static. To run the model in eager mode, set 'enforce_eager=True' or use '--enforce-eager' in the CLI.
INFO 07-30 01:15:06 model_runner.py:928] CUDA graphs can take additional 1~3 GiB memory per GPU. If you are running out of memory, consider decreasing `gpu_memory_utilization` or enforcing eager mode. You can also reduce the `max_num_seqs` as needed to decrease memory usage.
INFO 07-30 01:15:26 model_runner.py:1117] Graph capturing finished in 20 secs.
モデルの初期化が完了しました。


In [6]:
print("--- ステップ3 & 4: 問題生成と結果の集計 ---")
# GemmaのInstructモデルのテンプレートに合わせてプロンプトを整形
prompts = [
    f"<start_of_turn>user\n{UPWARD_EVOLUTION_PROMPT_TEMPLATE.format(problem=row['problem'])}<end_of_turn>\n<start_of_turn>model"
    for _, row in problems_to_process.iterrows()
]

start_time = time.time()
outputs = llm.generate(prompts, sampling_params)
end_time = time.time()
total_elapsed_time = end_time - start_time
print(f"問題生成が完了しました。処理時間: {total_elapsed_time:.2f}秒")

# 結果の集計
results = []
for i, output in enumerate(outputs):
    original_problem_text = problems_to_process.iloc[i]['problem']
    generated_text = output.outputs[0].text
    evolved_problem = parse_final_instruction(generated_text)
    avg_time_per_problem = total_elapsed_time / len(outputs) if len(outputs) > 0 else 0

    results.append({
        "original_problem": original_problem_text,
        "evolved_problem": evolved_problem,
        "total_tokens": len(output.outputs[0].token_ids),
        "elapsed_time_avg": avg_time_per_problem,
        "success": evolved_problem is not None,
        "full_model_output": generated_text
    })
print("結果の集計が完了しました。")

--- ステップ3 & 4: 問題生成と結果の集計 ---


Processed prompts: 100%|██████████| 100/100 [01:14<00:00,  1.34it/s, est. speed input: 700.61 toks/s, output: 698.94 toks/s]

問題生成が完了しました。処理時間: 75.06秒
結果の集計が完了しました。


In [7]:
print("--- ステップ5: 結果の保存とプレビュー ---")
results_df = pd.DataFrame(results)
output_filename = "results.csv"
results_df.to_csv(output_filename, index=False, encoding='utf-8-sig')
print(f"結果を'{output_filename}'に保存しました。")

# 結果のプレビューを表示
print("\n--- 生成データ プレビュー ---")
display(results_df.head())

--- ステップ5: 結果の保存とプレビュー ---
結果を'results.csv'に保存しました。

--- 生成データ プレビュー ---


,original_problem,evolved_problem,total_tokens,elapsed_time_avg,success,full_model_output
0,"\nA four-digit number, when a 7 is added in fr...","A four-digit prime number, when a 7 is added i...",393,0.750597,True,\n\nStep 1\n#Elements Identified#:\nOriginal i...
1,\nA student-entrepreneur bought several packs ...,A student-entrepreneur bought several packs of...,460,0.750597,True,\n\nStep 1\n#Elements Identified#:\n- A studen...
2,\nAn athlete weighing 78.75 kg tests a net use...,An athlete weighing 78.75 kg jumps from a heig...,525,0.750597,True,\n\nStep 1\n#Elements Identified#:\n- An athle...
3,\nBullies Jay and Bob drew tadpoles during dra...,Bullies Jay and Bob drew tadpoles during draft...,284,0.750597,True,\n\nStep 1\n#Elements Identified#:\nBullies Ja...
4,"\nBusinessmen Ivanov, Petrov, and Sidorov deci...",Ivanov and Petrov decided to create a car comp...,425,0.750597,True,\n\nI understand. Let's start by identifying t...


In [15]:
print("--- ステップ5: Hugging Face Hubへのアップロードとプレビュー ---")
try:
    # 結果をpandas DataFrameに変換
    results_df = pd.DataFrame(results)

    # pandas DataFrameからHugging Face Datasetオブジェクトに変換
    hf_dataset = Dataset.from_pandas(results_df)

    # !!修正!!: セル2で受け取ったトークンを直接指定してアップロード
    hf_dataset.push_to_hub(
        repo_id=OUTPUT_DATASET_ID,
        private=True, # 非公開データセットとして作成する場合はTrue
        token=hf_write_token
    )
    print(f"データセットを '{OUTPUT_DATASET_ID}' に正常にアップロードしました。")

    # 結果のプレビューを表示
    print("\n--- 生成データ プレビュー ---")
    display(results_df.head())

except Exception as e:
    print(f"Hugging Face Hubへのアップロードに失敗しました: {e}")



--- ステップ5: Hugging Face Hubへのアップロードとプレビュー ---


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)                : |          |  0.00B /  0.00B            

New Data Upload                         : |          |  0.00B /  0.00B            

                                        : 100%|##########|  118kB /  118kB            

データセットを 'Man-snow/test_evolved-math-problems' に正常にアップロードしました。

--- 生成データ プレビュー ---


,original_problem,evolved_problem,total_tokens,elapsed_time_avg,success,full_model_output
0,"\nA four-digit number, when a 7 is added in fr...","A four-digit prime number, when a 7 is added i...",393,0.750597,True,\n\nStep 1\n#Elements Identified#:\nOriginal i...
1,\nA student-entrepreneur bought several packs ...,A student-entrepreneur bought several packs of...,460,0.750597,True,\n\nStep 1\n#Elements Identified#:\n- A studen...
2,\nAn athlete weighing 78.75 kg tests a net use...,An athlete weighing 78.75 kg jumps from a heig...,525,0.750597,True,\n\nStep 1\n#Elements Identified#:\n- An athle...
3,\nBullies Jay and Bob drew tadpoles during dra...,Bullies Jay and Bob drew tadpoles during draft...,284,0.750597,True,\n\nStep 1\n#Elements Identified#:\nBullies Ja...
4,"\nBusinessmen Ivanov, Petrov, and Sidorov deci...",Ivanov and Petrov decided to create a car comp...,425,0.750597,True,\n\nI understand. Let's start by identifying t...
